In [4]:
!pip install pulp
!pip install requests

This model was generated to do an initial test on initial players to choose from FPL historical data

This needs to be added to the main FPL note book ultimately so there is one source of information

In [5]:
import requests
import pulp as lp

# Define the FPL API URL for player data
fpl_api_url = "https://fantasy.premierleague.com/api/bootstrap-static/"

# Make a request to the FPL API to fetch player data
response = requests.get(fpl_api_url)
data = response.json()

# Extract player data from the API response
players_data = {}
teams_data = {}  # Store team names
for player in data["elements"]:
    player_name = f"{player['first_name']} {player['second_name']}"
    players_data[player_name] = {
        "position": player["element_type"],
        "points_total": player["total_points"],
        "cost": player["now_cost"] / 10,  # Convert cost to millions
        "team": player["team"],
    }
    team_id = player["team"]
    team_name = data["teams"][team_id - 1]["name"]  # Adjust team_id to index
    teams_data[team_id] = team_name

# Define your total budget in millions
total_budget = 100  # Updated maximum budget to £100 million

# Define the maximum and minimum number of players allowed for each position
max_players_per_position = {
    1: 2,  # Goalkeeper
    2: 5,  # Defender
    3: 5,  # Midfielder
    4: 3,  # Forward (Striker)
}

min_players_per_position = {
    1: 2,  # Minimum 2 Goalkeeper
    2: 5,  # Minimum 5 Defenders
    3: 5,  # Minimum 5 Midfielders
    4: 3,  # Minimum 3 Forward (Striker)
}

# Define the minimum total players required
min_total_players = 15  # Updated minimum team size to 15 players

# Create a PuLP linear programming problem
model = lp.LpProblem("FantasyFootballTeam", lp.LpMaximize)

# Create a binary variable for each player to represent if they are in the team
player_vars = {player: lp.LpVariable(player, cat=lp.LpBinary) for player in players_data.keys()}

# Objective function: maximize total points
model += lp.lpSum(players_data[player]["points_total"] * player_vars[player] for player in players_data.keys()), "TotalPoints"

# Constraints
# Budget constraint
model += lp.lpSum(players_data[player]["cost"] * player_vars[player] for player in players_data.keys()) <= total_budget, "Budget"

# Position constraints
for position, max_count in max_players_per_position.items():
    model += lp.lpSum(player_vars[player] for player in players_data.keys() if players_data[player]["position"] == position) <= max_count, f"MaxPlayersPosition{position}"

# Minimum players per position constraints
for position, min_count in min_players_per_position.items():
    model += lp.lpSum(player_vars[player] for player in players_data.keys() if players_data[player]["position"] == position) >= min_count, f"MinPlayersPosition{position}"

# Minimum total players constraint
model += lp.lpSum(player_vars[player] for player in players_data.keys()) >= min_total_players, "MinTotalPlayers"

# Solve the ILP problem
model.solve()

# Create a dictionary to store selected players by position
selected_players_by_position = {
    1: [],  # Goalkeeper
    2: [],  # Defender
    3: [],  # Midfielder
    4: [],  # Forward (Striker)
}

# Populate the dictionary with selected players
for player in players_data.keys():
    if lp.value(player_vars[player]) > 0:
        data = players_data[player]
        selected_players_by_position[data["position"]].append(player)

# Print the selected players and their data sorted by position
for position in selected_players_by_position.keys():
    position_name = {
        1: "Goalkeeper",
        2: "Defender",
        3: "Midfielder",
        4: "Forward (Striker)",
    }.get(position, "Unknown")
    print(f"Position: {position_name}")
    for player in selected_players_by_position[position]:
        data = players_data[player]
        team_name = teams_data[data["team"]]  # Get team name from teams_data
        print(f"Player: {player} - Points: {data['points_total']} - Cost: £{data['cost']} million - Team: {team_name}")

# Suggest the captain based on the player with the highest points
captain = None
max_points = 0
for player in players_data.keys():
    if lp.value(player_vars[player]) > 0:
        if players_data[player]["points_total"] > max_points:
            captain = player
            max_points = players_data[player]["points_total"]

if captain:
    print(f"\nSuggested Captain (Double Points): {captain}")

# You can customize the constraints and objective function as needed

# Generate total points and cost of the selected team
total_points = sum(players_data[player]["points_total"] for player in players_data.keys() if lp.value(player_vars[player]) > 0)
total_cost = sum(players_data[player]["cost"] for player in players_data.keys() if lp.value(player_vars[player]) > 0)
print(f"\nTotal Points: {total_points}")
print(f"Total Cost: £{total_cost} million")



Position: Goalkeeper
Player: David Raya Martin - Points: 135 - Cost: £5.3 million - Team: Arsenal
Player: Jordan Pickford - Points: 153 - Cost: £4.8 million - Team: Everton
Position: Defender
Player: Gabriel dos Santos Magalhães - Points: 149 - Cost: £5.4 million - Team: Arsenal
Player: William Saliba - Points: 164 - Cost: £5.9 million - Team: Arsenal
Player: Benjamin White - Points: 182 - Cost: £6.1 million - Team: Arsenal
Player: Jarrad Branthwaite - Points: 124 - Cost: £4.5 million - Team: Everton
Player: Pedro Porro - Points: 136 - Cost: £5.9 million - Team: Spurs
Position: Midfielder
Player: Bukayo Saka - Points: 226 - Cost: £8.8 million - Team: Arsenal
Player: Cole Palmer - Points: 244 - Cost: £6.3 million - Team: Chelsea
Player: Phil Foden - Points: 230 - Cost: £8.5 million - Team: Man City
Player: Anthony Gordon - Points: 183 - Cost: £6.4 million - Team: Newcastle
Player: Son Heung-min - Points: 213 - Cost: £10.0 million - Team: Spurs
Position: Forward (Striker)
Player: Ollie W